In [1]:
from ALLCools.mcds import RegionDS
from ALLCools.dmr import call_dms, call_dmr
import glob as glob
import pandas as pd
import numpy as np
import os

In [2]:
brain_regions = ['PFC','AMY','HIP']

In [3]:
cell_types = ['Neuron','Astro-Epen','Vascular','Immune','OPC-Oligo']

In [4]:
celltype = cell_types[0]
brain_region = brain_regions[0]

In [5]:
mc_bulk_dir = f'/data2st1/junyi/merged_allcs/{brain_region}/'
allc_paths = glob.glob(f'{mc_bulk_dir}/*.tsv.gz')

In [6]:
# 3rd column is the orginal id in the wilcox table
# df_dar = pd.read_csv(f'/data2st1/junyi/output/motif/ALL_{celltype}_wilcoxon_DAR_lifted_sorted_gene.bed',sep='\t',header=None,index_col=3)
# df_dar.columns = ['chr','start','end','chrmm10','startmm10','endmm10','score','strand','gene_name','gene_id','region','distance']
# df_pval = pd.read_csv(f'/data2st1/junyi/output/motif/ALL_{celltype}_wilcoxon.csv',index_col=0)
# df_concatted= pd.concat([df_pval.loc[df_dar.index],df_dar],axis=1)
# df_dar_selected = df_concatted[(df_concatted.pvals<0.05) & (df_concatted.logfoldchanges>0.1)& (df_concatted.pct_nz_group>0.1)]
# df_dar_selected['regionmm10'] = df_dar_selected['chr'] + ':' + df_dar_selected['start'].astype(str) + '-' + df_dar_selected['end'].astype(str)
# df_dar_selected['region_extended'] = df_dar_selected['chr'] + ':' + (df_dar_selected['start']-5000).astype(str) + '-' + (df_dar_selected['end']+5000).astype(str)

In [7]:
df_dar = pd.read_csv('/data1st2/hannan_25/data/Nanopore_process/nanopore_08_differential/pair_base/C_dmrpair_F_HIP_vs_PFC.segment',sep='\t',header=None,index_col=None)
df_dar.columns = ['chr','start','end','state','scores','nsties','counta','couontb','pcta','pctb','fraca','fracb','effectsize']
df_dar_selected = df_dar[(df_dar['state'].isin(['different'])) & (df_dar['chr'].str.len()<=2) & (df_dar['fracb']>0.1) & (df_dar['effectsize']<0)]
df_dar_selected['regionmm10'] = 'chr'+df_dar_selected['chr'] + ':' + df_dar_selected['start'].astype(str) + '-' + df_dar_selected['end'].astype(str)
df_dar_selected['region_extended'] = 'chr'+df_dar_selected['chr'] + ':' + (df_dar_selected['start']-5000).astype(str) + '-' + (df_dar_selected['end']+5000).astype(str)

/tmp/ipykernel_1474198/1512998197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dar_selected['regionmm10'] = 'chr'+df_dar_selected['chr'] + ':' + df_dar_selected['start'].astype(str) + '-' + df_dar_selected['end'].astype(str)
/tmp/ipykernel_1474198/1512998197.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dar_selected['region_extended'] = 'chr'+df_dar_selected['chr'] + ':' + (df_dar_selected['start']-5000).astype(str) + '-' + (df_dar_selected['end']+5000).astype(str)


In [8]:
df_dar_selected.sort_values(by='scores',ascending=False)

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,fraca,fracb,effectsize,regionmm10,region_extended
19103,5,116478887,116478906,different,168.399065,19,"h:69,m:220","h:229,m:160","h:12.15,m:38.73","h:55.85,m:39.02",0.508803,0.948781,-0.439978,chr5:116478887-116478906,chr5:116473887-116483906
6285,13,78184643,78184841,different,159.601391,93,"h:42,m:141","h:168,m:298","h:0.96,m:3.22","h:5.92,m:10.51",0.041771,0.164316,-0.122545,chr13:78184643-78184841,chr13:78179643-78189841
39052,Y,90794037,90794067,different,153.709346,27,"h:121,m:8331","h:133,m:3894","h:1.17,m:80.77","h:3.04,m:88.88",0.819389,0.919197,-0.099807,chrY:90794037-90794067,chrY:90789037-90799067
13512,2,98666709,98666771,different,151.803719,25,"h:521,m:33229","h:712,m:21851","h:0.42,m:26.49","h:0.93,m:28.45",0.269016,0.293808,-0.024792,chr2:98666709-98666771,chr2:98661709-98671771
16988,4,87432291,87432329,different,151.436004,38,"h:6,m:534","h:6,m:580","h:0.54,m:48.50","h:0.90,m:87.35",0.490463,0.882530,-0.392067,chr4:87432291-87432329,chr4:87427291-87437329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29740,9,49151886,49151925,different,-0.427192,23,"h:6,m:591","h:2,m:199","h:0.70,m:69.37","h:0.72,m:71.58",0.700704,0.723022,-0.022317,chr9:49151886-49151925,chr9:49146886-49156925
34306,X,38315856,38315858,different,-0.457143,2,"h:0,m:23","h:0,m:7","h:0.00,m:47.92","h:0.00,m:58.33",0.479167,0.583333,-0.104167,chrX:38315856-38315858,chrX:38310856-38320858
35469,X,77414722,77414751,different,-0.572276,17,"h:14,m:228","h:7,m:91","h:2.41,m:39.18","h:3.00,m:39.06",0.415808,0.420601,-0.004793,chrX:77414722-77414751,chrX:77409722-77419751
38806,Y,90740368,90740376,different,-0.618455,5,"h:13,m:475","h:6,m:236","h:0.87,m:31.65","h:0.82,m:32.42",0.325117,0.332418,-0.007301,chrY:90740368-90740376,chrY:90735368-90745376


In [9]:
df_dar_selected

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,fraca,fracb,effectsize,regionmm10,region_extended
1,1,3064927,3064929,different,26.685243,2,"h:0,m:0","h:0,m:19","h:0.00,m:0.00","h:0.00,m:48.72",0.000000,0.487179,-0.487179,chr1:3064927-3064929,chr1:3059927-3069929
3,1,3734993,3734995,different,37.454968,2,"h:0,m:1","h:0,m:37","h:0.00,m:2.70","h:0.00,m:92.50",0.027027,0.925000,-0.897973,chr1:3734993-3734995,chr1:3729993-3739995
19,1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",0.030973,0.302326,-0.271352,chr1:6526617-6526633,chr1:6521617-6531633
21,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6527054-6527089,chr1:6522054-6532089
27,1,7026106,7026107,different,18.837102,1,"h:0,m:0","h:0,m:9","h:0.00,m:0.00","h:0.00,m:90.00",0.000000,0.900000,-0.900000,chr1:7026106-7026107,chr1:7021106-7031107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,Y,90807380,90807382,different,30.930988,2,"h:0,m:18","h:0,m:36","h:0.00,m:3.05","h:0.00,m:25.17",0.030457,0.251748,-0.221291,chrY:90807380-90807382,chrY:90802380-90812382
39128,Y,90807414,90807415,different,35.765283,1,"h:0,m:2","h:2,m:19","h:0.00,m:0.47","h:2.70,m:25.68",0.004739,0.283784,-0.279044,chrY:90807414-90807415,chrY:90802414-90812415
39132,Y,90807806,90807826,different,14.077198,10,"h:67,m:769","h:78,m:342","h:1.63,m:18.73","h:3.96,m:17.34",0.203604,0.212982,-0.009377,chrY:90807806-90807826,chrY:90802806-90812826
39134,Y,90808876,90808878,different,9.732585,2,"h:38,m:446","h:40,m:185","h:4.71,m:55.33","h:12.08,m:55.89",0.600496,0.679758,-0.079262,chrY:90808876-90808878,chrY:90803876-90813878


In [10]:
allc_table= {}

for allc_path in cell_types:
    #allc_table["/data2st1/junyi/merged_allcs/PFC/"+allc_path] = mc_bulk_dir+allc_path+'.tsv.gz'
    allc_table[allc_path] = mc_bulk_dir+allc_path+'.tsv.gz'


In [11]:
allc_table

{'Neuron': '/data2st1/junyi/merged_allcs/PFC/Neuron.tsv.gz',
 'Astro-Epen': '/data2st1/junyi/merged_allcs/PFC/Astro-Epen.tsv.gz',
 'Vascular': '/data2st1/junyi/merged_allcs/PFC/Vascular.tsv.gz',
 'Immune': '/data2st1/junyi/merged_allcs/PFC/Immune.tsv.gz',
 'OPC-Oligo': '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo.tsv.gz'}

In [12]:
# make a dict, key is sample name, value is allc path
# import pathlib
# allc_table = {
#     allc_path.name.split('.')[0]: str(allc_path)
#     for allc_path in pathlib.Path(mc_bulk_dir).glob(
#         f'{mc_bulk_dir}/*.tsv.gz')
# }
celltype = 'Nano_PFC_HIP'
samples = list(allc_table.keys())
allc_paths = list(allc_table.values())
chrom_size_path = '/home/junyichen/code/whole_mouse_brain/wmb/files/mm10.main.chrom.sizes'
output_dir = f'/data2st1/junyi/output/dmr/{brain_region}/{celltype}/'


In [13]:
if os.path.exists(output_dir) == False:
    os.makedirs(output_dir)

In [16]:
input_region = df_dar_selected.sort_values(by='scores',ascending=False)
input_region.sort_values(['chr','start','end'],inplace=True)
input_region.drop_duplicates(subset=['chr','start','end'],keep='first',inplace=True)
df_top = input_region
input_region = input_region.regionmm10.values.tolist()  

In [17]:
input_region

['chr1:3064927-3064929',
 'chr1:3734993-3734995',
 'chr1:6526617-6526633',
 'chr1:6527054-6527089',
 'chr1:7026106-7026107',
 'chr1:11623891-11623893',
 'chr1:12577161-12577177',
 'chr1:12653555-12653556',
 'chr1:14294344-14294350',
 'chr1:15724206-15724207',
 'chr1:15995719-15995722',
 'chr1:16878736-16878786',
 'chr1:16878858-16878870',
 'chr1:17077820-17077822',
 'chr1:17944447-17944451',
 'chr1:18098940-18099022',
 'chr1:20410347-20410351',
 'chr1:21041957-21041959',
 'chr1:22319206-22319208',
 'chr1:24212402-24212453',
 'chr1:24268482-24268485',
 'chr1:24615077-24615078',
 'chr1:26615837-26615838',
 'chr1:27481459-27481551',
 'chr1:31782475-31782489',
 'chr1:33638151-33638154',
 'chr1:34124605-34124607',
 'chr1:34124654-34124660',
 'chr1:34124761-34124768',
 'chr1:35909386-35909389',
 'chr1:36416015-36416040',
 'chr1:36683219-36683220',
 'chr1:37100223-37100244',
 'chr1:37667467-37667537',
 'chr1:37820063-37820082',
 'chr1:38634732-38634744',
 'chr1:39678835-39678839',
 'chr1:4002

In [18]:
call_dms(
    output_dir=output_dir,
    allc_paths=allc_paths,
    samples=samples,
    chrom_size_path=chrom_size_path,
    cpu=64,
    max_row_count=50,
    n_permute=3000,
    min_pvalue=0.01,
    # here we just calculate some small regions for demo
    # do not provide region parameter if you want to run DMR calling for the whole genome
    # This parameter can also be used for call DMR/DMS in specific region of interest
    region=input_region)


RMS tests for 21 sites.
RMS tests for 6 sites.RMS tests for 2 sites.RMS tests for 2 sites.


RMS tests for 15 sites.
RMS tests for 6 sites.RMS tests for 2 sites.

RMS tests for 14 sites.
RMS tests for 2 sites.RMS tests for 18 sites.

RMS tests for 4 sites.RMS tests for 13 sites.

RMS tests for 2 sites.
RMS tests for 28 sites.
RMS tests for 10 sites.RMS tests for 10 sites.

RMS tests for 12 sites.
RMS tests for 5 sites.RMS tests for 6 sites.RMS tests for 24 sites.


RMS tests for 12 sites.
RMS tests for 8 sites.
RMS tests for 13 sites.
RMS tests for 3 sites.
RMS tests for 6 sites.
RMS tests for 4 sites.
RMS tests for 1 sites.
RMS tests for 6 sites.RMS tests for 12 sites.

RMS tests for 2 sites.
RMS tests for 2 sites.
RMS tests for 3 sites.RMS tests for 7 sites.

RMS tests for 14 sites.
RMS tests for 7 sites.
RMS tests for 4 sites.RMS tests for 1 sites.

RMS tests for 32 sites.
RMS tests for 6 sites.
RMS tests for 3 sites.
RMS tests for 3 sites.RMS tests for 37 sites.

RMS tests for 2 si

In [19]:
call_dmr(output_dir=output_dir,
         p_value_cutoff=0.001,
         frac_delta_cutoff=0.3,
         max_dist=250,
         residual_quantile=0.7,
         corr_cutoff=0.3,
         cpu=64)

In [20]:
dmr_ds = RegionDS.open(output_dir)
dmr_ds


Using dmr as region_dim


<xarray.RegionDS>
Dimensions:      (count_type: 2, dmr: 1073, sample: 5)
Coordinates:
  * count_type   (count_type) <U3 'mc' 'cov'
  * dmr          (dmr) <U8 'chr1-0' 'chr1-1' 'chr1-2' ... 'chrX-1' 'chrX-2'
    dmr_chrom    (dmr) <U5 dask.array<chunksize=(214,), meta=np.ndarray>
    dmr_end      (dmr) int64 dask.array<chunksize=(214,), meta=np.ndarray>
    dmr_length   (dmr) int64 dask.array<chunksize=(214,), meta=np.ndarray>
    dmr_ndms     (dmr) int64 dask.array<chunksize=(214,), meta=np.ndarray>
    dmr_start    (dmr) int64 dask.array<chunksize=(214,), meta=np.ndarray>
  * sample       (sample) <U10 'Neuron' 'Astro-Epen' ... 'Immune' 'OPC-Oligo'
Data variables:
    dmr_da       (sample, dmr, count_type) uint32 dask.array<chunksize=(1, 214, 1), meta=np.ndarray>
    dmr_da_frac  (sample, dmr) float32 dask.array<chunksize=(1, 214), meta=np.ndarray>
    dmr_state    (sample, dmr) int8 dask.array<chunksize=(1, 214), meta=np.ndarray>
Attributes:
    region_dim:          dmr
    region_ds_location:  /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP
    chrom_size_path:     /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP/chrom_si...

In [21]:
df_dmr = pd.DataFrame(
    {
        'chr': dmr_ds['dmr_chrom'].to_numpy(),
        'start': dmr_ds['dmr_start'].to_numpy(),
        'end': dmr_ds['dmr_end'].to_numpy(),
    }
)

In [22]:
sample_mapping = {}
for i in range(len(dmr_ds['sample'].to_numpy())):
    sample_mapping[i] = dmr_ds['sample'].to_numpy()[i]
sample_mapping

{0: 'Neuron', 1: 'Astro-Epen', 2: 'Vascular', 3: 'Immune', 4: 'OPC-Oligo'}

In [23]:
drm_state = dmr_ds['dmr_state'].to_numpy()
for i in range(len(drm_state)):
    df_dmr[sample_mapping[i]] = drm_state[i]

In [ ]:
df_dmr.to_csv(f'{output_dir}/dmr_stat.csv',index=False)

In [34]:
df_dmr

,chr,start,end,Neuron,Astro-Epen,Vascular,Immune,OPC-Oligo
0,chr1,6527054,6527090,1,0,0,0,-1
1,chr1,12577161,12577178,1,0,0,0,-1
2,chr1,16878736,16878787,1,-1,0,0,-1
3,chr1,18098985,18099023,1,0,0,0,0
4,chr1,24212402,24212421,1,-1,0,0,-1
...,...,...,...,...,...,...,...,...
1068,chr19,58245547,58245607,1,0,0,0,0
1069,chr19,59249305,59249307,1,-1,0,-1,0
1070,chrX,12195699,12195728,1,-1,0,-1,0
1071,chrX,103592125,103592127,1,-1,0,-1,1


In [33]:
df_dar_selected

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,fraca,fracb,effectsize,regionmm10,region_extended
1,1,3064927,3064929,different,26.685243,2,"h:0,m:0","h:0,m:19","h:0.00,m:0.00","h:0.00,m:48.72",0.000000,0.487179,-0.487179,chr1:3064927-3064929,chr1:3059927-3069929
3,1,3734993,3734995,different,37.454968,2,"h:0,m:1","h:0,m:37","h:0.00,m:2.70","h:0.00,m:92.50",0.027027,0.925000,-0.897973,chr1:3734993-3734995,chr1:3729993-3739995
19,1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",0.030973,0.302326,-0.271352,chr1:6526617-6526633,chr1:6521617-6531633
21,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6527054-6527089,chr1:6522054-6532089
27,1,7026106,7026107,different,18.837102,1,"h:0,m:0","h:0,m:9","h:0.00,m:0.00","h:0.00,m:90.00",0.000000,0.900000,-0.900000,chr1:7026106-7026107,chr1:7021106-7031107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,Y,90807380,90807382,different,30.930988,2,"h:0,m:18","h:0,m:36","h:0.00,m:3.05","h:0.00,m:25.17",0.030457,0.251748,-0.221291,chrY:90807380-90807382,chrY:90802380-90812382
39128,Y,90807414,90807415,different,35.765283,1,"h:0,m:2","h:2,m:19","h:0.00,m:0.47","h:2.70,m:25.68",0.004739,0.283784,-0.279044,chrY:90807414-90807415,chrY:90802414-90812415
39132,Y,90807806,90807826,different,14.077198,10,"h:67,m:769","h:78,m:342","h:1.63,m:18.73","h:3.96,m:17.34",0.203604,0.212982,-0.009377,chrY:90807806-90807826,chrY:90802806-90812826
39134,Y,90808876,90808878,different,9.732585,2,"h:38,m:446","h:40,m:185","h:4.71,m:55.33","h:12.08,m:55.89",0.600496,0.679758,-0.079262,chrY:90808876-90808878,chrY:90803876-90813878


In [41]:
dmr_ds['dmr_da_frac'].to_numpy()

array([[0.70711297, 0.82464457, 0.93630576, ..., 0.88188976, 1.        ,
        0.88461536],
       [0.21212122, 0.        , 0.3       , ..., 0.        , 0.        ,
        0.16666667],
       [0.07142857, 0.15      , 0.75      , ..., 1.        , 0.44444445,
        0.33333334],
       [0.22222222, 0.375     , 1.        , ..., 0.05      , 0.        ,
        0.        ],
       [0.06451613, 0.        , 0.4       , ..., 0.5555556 , 1.        ,
        0.25      ]], dtype=float32)

In [26]:
np.save(f'{output_dir}/dmr_frac.npy', dmr_ds['dmr_da_frac'].to_numpy())


In [28]:
NN_marker = dmr_ds['dmr_state'][0]==-1

In [29]:
dmr_nn = dmr_ds.sel(dmr=NN_marker.to_numpy())

In [30]:
dmr_nn['dmr_da_frac'].to_numpy()[:,0]

array([0.31226766, 0.74285716, 0.9       , 0.9375    , 0.34666666],
      dtype=float32)

In [186]:
dmr_nn['dmr_da_frac'].to_numpy()[:,1]

array([0.36842105, 1.        , 1.        , 1.        , 0.6363636 ],
      dtype=float32)

In [211]:
dmr_nn['dmr_start'].to_numpy()

array([ 90486466, 132730271,  59350487,  82964798,  69564613,  69572318,
        46822596])

In [212]:
dmr_nn['sample']

<xarray.DataArray 'sample' (sample: 5)>
array(['Neuron', 'Astro-Epen', 'Vascular', 'Immune', 'OPC-Oligo'], dtype='<U10')
Coordinates:
  * sample   (sample) <U10 'Neuron' 'Astro-Epen' 'Vascular' 'Immune' 'OPC-Oligo'

In [207]:
dmr_nn['dmr_da_frac'].to_numpy()[:,:].round(3)

array([[0.224, 0.368, 0.544, 0.125, 0.4  , 0.557, 0.533],
       [0.68 , 1.   , 0.933, 0.   , 0.5  , 0.96 , 1.   ],
       [0.7  , 1.   , 0.833, 1.   , 1.   , 1.   , 1.   ],
       [1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ],
       [0.37 , 0.636, 1.   , 0.714, 1.   , 0.99 , 1.   ]], dtype=float32)

In [197]:
dmr_nn['dmr_da'].to_numpy()[:,:,0]

array([[ 24,  21,  31,   3,  14, 172,  24],
       [ 17,  11,  14,   0,   5,  48,  10],
       [  7,   4,   5,   3,   5,  16,   4],
       [ 10,   5,   3,   7,   2,  18,   4],
       [ 10,   7,  12,   5,  11,  96,   5]], dtype=uint32)

In [198]:
dmr_nn['dmr_da'].to_numpy()[:,:,1]

array([[107,  57,  57,  24,  35, 309,  45],
       [ 25,  11,  15,  10,  10,  50,  10],
       [ 10,   4,   6,   3,   5,  16,   4],
       [ 10,   5,   3,   7,   2,  18,   4],
       [ 27,  11,  12,   7,  11,  97,   5]], dtype=uint32)

In [36]:
dms_ds = RegionDS.open(output_dir, region_dim='dms')
dms_ds


<xarray.RegionDS>
Dimensions:       (count_type: 2, dms: 3912, sample: 5)
Coordinates:
  * count_type    (count_type) <U3 'mc' 'cov'
  * dms           (dms) <U15 'chr1-6526628' 'chr1-6527055' ... 'chrX-162898192'
    dms_chrom     (dms) <U5 dask.array<chunksize=(782,), meta=np.ndarray>
    dms_contexts  (dms) <U3 dask.array<chunksize=(782,), meta=np.ndarray>
    dms_p-values  (dms) float64 dask.array<chunksize=(782,), meta=np.ndarray>
    dms_pos       (dms) int64 dask.array<chunksize=(782,), meta=np.ndarray>
  * sample        (sample) <U10 'Neuron' 'Astro-Epen' ... 'Immune' 'OPC-Oligo'
Data variables:
    dms_da        (sample, dms, count_type) float64 dask.array<chunksize=(1, 782, 1), meta=np.ndarray>
    dms_da_frac   (sample, dms) float64 dask.array<chunksize=(1, 782), meta=np.ndarray>
    dms_residual  (sample, dms) float64 dask.array<chunksize=(1, 782), meta=np.ndarray>
Attributes:
    region_dim:          dms
    region_ds_location:  /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP
    chrom_size_path:     /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP/chrom_si...

In [205]:
dms_ds['dms_residual'].to_numpy().shape

(5, 943)

In [216]:
dms_ds

<xarray.RegionDS>
Dimensions:       (count_type: 2, dms: 3912, sample: 5)
Coordinates:
  * count_type    (count_type) <U3 'mc' 'cov'
  * dms           (dms) <U15 'chr1-6526628' 'chr1-6527055' ... 'chrX-162898192'
    dms_chrom     (dms) <U5 dask.array<chunksize=(782,), meta=np.ndarray>
    dms_contexts  (dms) <U3 dask.array<chunksize=(782,), meta=np.ndarray>
    dms_p-values  (dms) float64 dask.array<chunksize=(782,), meta=np.ndarray>
    dms_pos       (dms) int64 dask.array<chunksize=(782,), meta=np.ndarray>
  * sample        (sample) <U10 'Neuron' 'Astro-Epen' ... 'Immune' 'OPC-Oligo'
Data variables:
    dms_da        (sample, dms, count_type) float64 dask.array<chunksize=(1, 782, 1), meta=np.ndarray>
    dms_da_frac   (sample, dms) float64 dask.array<chunksize=(1, 782), meta=np.ndarray>
    dms_residual  (sample, dms) float64 dask.array<chunksize=(1, 782), meta=np.ndarray>
Attributes:
    region_dim:          dms
    region_ds_location:  /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP
    chrom_size_path:     /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP/chrom_si...

In [ ]:
dms_dsstat= dms_ds['dms'].to_dataframe()
dms_mc = dms_ds['dms_da'][:,:,0].to_numpy().T
df_mc = pd.DataFrame(dms_mc,columns=dms_ds['sample'].to_numpy())
df_mc.columns = ['mc_'+str(i) for i in df_mc.columns]
dms_cov = dms_ds['dms_da'][:,:,1].to_numpy().T
df_cov = pd.DataFrame(dms_cov,columns=dms_ds['sample'].to_numpy())
#df_cov.columns = ['cov'+str(i) for i in range(df_cov.shape[1])]
df_cov.columns= ['cov_'+str(i) for i in df_cov.columns]
df_cov_mc = pd.concat([df_mc,df_cov],axis=1)
df_cov_mc['dms']= dms_dsstat['dms'].values
df_cov_mc.set_index('dms',inplace=True)
dms_stat_cov = dms_dsstat.merge(df_cov_mc,left_index=True,right_index=True)

In [140]:
dms_stat_cov.head(3)

,dms_chrom,dms_contexts,dms_p-values,dms_pos,dms,mc_Neuron,mc_Astro-Epen,mc_Vascular,mc_Immune,mc_OPC-Oligo,cov_Neuron,cov_Astro-Epen,cov_Vascular,cov_Immune,cov_OPC-Oligo
dms,,,,,,,,,,,,,,,
chr1-6526628,chr1,CAA,0.004333,6526628,chr1-6526628,19.0,0.0,0.0,0.0,0.0,49.0,9.0,3.0,1.0,12.0
chr1-6527055,chr1,CAT,0.000333,6527055,chr1-6527055,32.0,0.0,0.0,0.0,0.0,81.0,10.0,8.0,7.0,20.0
chr1-6527080,chr1,CTC,0.005000,6527080,chr1-6527080,21.0,0.0,0.0,0.0,0.0,55.0,9.0,8.0,3.0,6.0


In [151]:
dms_stat_cov

,dms_chrom,dms_contexts,dms_p-values,dms_pos,dms,mc_Neuron,mc_Astro-Epen,mc_Vascular,mc_Immune,mc_OPC-Oligo,cov_Neuron,cov_Astro-Epen,cov_Vascular,cov_Immune,cov_OPC-Oligo
dms,,,,,,,,,,,,,,,
chr1-6526628,chr1,CAA,0.004333,6526628,chr1-6526628,19.0,0.0,0.0,0.0,0.0,49.0,9.0,3.0,1.0,12.0
chr1-6527055,chr1,CAT,0.000333,6527055,chr1-6527055,32.0,0.0,0.0,0.0,0.0,81.0,10.0,8.0,7.0,20.0
chr1-6527080,chr1,CTC,0.005000,6527080,chr1-6527080,21.0,0.0,0.0,0.0,0.0,55.0,9.0,8.0,3.0,6.0
chr1-6527082,chr1,CAT,0.000333,6527082,chr1-6527082,64.0,0.0,0.0,0.0,0.0,82.0,11.0,10.0,5.0,22.0
chr1-6527089,chr1,CGG,0.000333,6527089,chr1-6527089,73.0,7.0,2.0,4.0,4.0,76.0,12.0,10.0,6.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX-12195727,chrX,CGG,0.000333,12195727,chrX-12195727,42.0,0.0,3.0,0.0,6.0,44.0,7.0,3.0,6.0,7.0
chrX-73794972,chrX,CGT,0.003667,73794972,chrX-73794972,6.0,5.0,0.0,2.0,3.0,26.0,6.0,0.0,3.0,5.0
chrX-103592126,chrX,CGG,0.000333,103592126,chrX-103592126,18.0,0.0,4.0,0.0,5.0,18.0,3.0,9.0,4.0,5.0


In [146]:
df_dar_selected

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,fraca,fracb,effectsize,regionmm10,region_extended
1,1,3064927,3064929,different,26.685243,2,"h:0,m:0","h:0,m:19","h:0.00,m:0.00","h:0.00,m:48.72",0.000000,0.487179,-0.487179,chr1:3064927-3064929,chr1:3059927-3069929
3,1,3734993,3734995,different,37.454968,2,"h:0,m:1","h:0,m:37","h:0.00,m:2.70","h:0.00,m:92.50",0.027027,0.925000,-0.897973,chr1:3734993-3734995,chr1:3729993-3739995
19,1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",0.030973,0.302326,-0.271352,chr1:6526617-6526633,chr1:6521617-6531633
21,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6527054-6527089,chr1:6522054-6532089
27,1,7026106,7026107,different,18.837102,1,"h:0,m:0","h:0,m:9","h:0.00,m:0.00","h:0.00,m:90.00",0.000000,0.900000,-0.900000,chr1:7026106-7026107,chr1:7021106-7031107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39126,Y,90807380,90807382,different,30.930988,2,"h:0,m:18","h:0,m:36","h:0.00,m:3.05","h:0.00,m:25.17",0.030457,0.251748,-0.221291,chrY:90807380-90807382,chrY:90802380-90812382
39128,Y,90807414,90807415,different,35.765283,1,"h:0,m:2","h:2,m:19","h:0.00,m:0.47","h:2.70,m:25.68",0.004739,0.283784,-0.279044,chrY:90807414-90807415,chrY:90802414-90812415
39132,Y,90807806,90807826,different,14.077198,10,"h:67,m:769","h:78,m:342","h:1.63,m:18.73","h:3.96,m:17.34",0.203604,0.212982,-0.009377,chrY:90807806-90807826,chrY:90802806-90812826
39134,Y,90808876,90808878,different,9.732585,2,"h:38,m:446","h:40,m:185","h:4.71,m:55.33","h:12.08,m:55.89",0.600496,0.679758,-0.079262,chrY:90808876-90808878,chrY:90803876-90813878


In [149]:
import pyranges as pr

# Example data (chr, start, end)
sites = pr.from_dict({"Chromosome": dms_stat_cov.dms_chrom, "Start": dms_stat_cov.dms_pos, "End": dms_stat_cov.dms_pos})
regions = pr.from_dict({"Chromosome": "chr"+df_dar_selected.chr, "Start": df_dar_selected.start, "End": df_dar_selected.end})

# Find overlaps
result = sites.join(regions)


In [158]:
result_df = result.df

In [169]:
result_df

,Chromosome,Start,End,Start_b,End_b
0,chr1,6526628,6526628,6526617,6526633
1,chr1,6527055,6527055,6527054,6527089
2,chr1,6527080,6527080,6527054,6527089
3,chr1,6527082,6527082,6527054,6527089
4,chr1,12577162,12577162,12577161,12577177
...,...,...,...,...,...
3224,chrX,12195727,12195727,12195699,12195733
3225,chrX,73794972,73794972,73794922,73794975
3226,chrX,103592126,103592126,103592123,103592127
3227,chrX,162898184,162898184,162898181,162898205


In [175]:
result_df['Chromosome'] = result_df['Chromosome'].astype(str)

In [200]:
result_df['dms'] = result_df['Chromosome']+"-"+result_df['Start'].astype(str)
result_df['regionmm10'] = result_df['Chromosome']+":"+result_df['Start_b'].astype(str)+"-"+result_df['End_b'].astype(str)

In [199]:
dms_stat_cov_selected = dms_stat_cov.loc[result_df['dms']]


In [205]:
df_dar_dms = df_dar_selected.set_index('regionmm10').loc[result_df['regionmm10']]

In [209]:
df_dar_dms

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,fraca,fracb,effectsize,region_extended
regionmm10,,,,,,,,,,,,,,
chr1:6526617-6526633,1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",0.030973,0.302326,-0.271352,chr1:6521617-6531633
chr1:6527054-6527089,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6522054-6532089
chr1:6527054-6527089,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6522054-6532089
chr1:6527054-6527089,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6522054-6532089
chr1:12577161-12577177,1,12577161,12577177,different,31.539541,11,"h:0,m:10","h:21,m:47","h:0.00,m:2.96","h:6.54,m:14.64",0.029586,0.211838,-0.182252,chr1:12572161-12582177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX:12195699-12195733,X,12195699,12195733,different,12.386906,21,"h:21,m:106","h:9,m:80","h:3.57,m:18.00","h:3.93,m:34.93",0.215620,0.388646,-0.173027,chrX:12190699-12200733
chrX:73794922-73794975,X,73794922,73794975,different,6.122430,28,"h:13,m:87","h:5,m:80","h:1.96,m:13.12","h:1.38,m:22.10",0.150830,0.234807,-0.083977,chrX:73789922-73799975
chrX:103592123-103592127,X,103592123,103592127,different,23.112939,4,"h:19,m:26","h:8,m:38","h:16.81,m:23.01","h:15.69,m:74.51",0.398230,0.901961,-0.503731,chrX:103587123-103597127


In [210]:
dms_stat_cov_selected

,dms_chrom,dms_contexts,dms_p-values,dms_pos,dms,mc_Neuron,mc_Astro-Epen,mc_Vascular,mc_Immune,mc_OPC-Oligo,cov_Neuron,cov_Astro-Epen,cov_Vascular,cov_Immune,cov_OPC-Oligo
dms,,,,,,,,,,,,,,,
chr1-6526628,chr1,CAA,0.004333,6526628,chr1-6526628,19.0,0.0,0.0,0.0,0.0,49.0,9.0,3.0,1.0,12.0
chr1-6527055,chr1,CAT,0.000333,6527055,chr1-6527055,32.0,0.0,0.0,0.0,0.0,81.0,10.0,8.0,7.0,20.0
chr1-6527080,chr1,CTC,0.005000,6527080,chr1-6527080,21.0,0.0,0.0,0.0,0.0,55.0,9.0,8.0,3.0,6.0
chr1-6527082,chr1,CAT,0.000333,6527082,chr1-6527082,64.0,0.0,0.0,0.0,0.0,82.0,11.0,10.0,5.0,22.0
chr1-12577162,chr1,CGA,0.000333,12577162,chr1-12577162,36.0,0.0,1.0,0.0,0.0,43.0,5.0,4.0,2.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX-12195727,chrX,CGG,0.000333,12195727,chrX-12195727,42.0,0.0,3.0,0.0,6.0,44.0,7.0,3.0,6.0,7.0
chrX-73794972,chrX,CGT,0.003667,73794972,chrX-73794972,6.0,5.0,0.0,2.0,3.0,26.0,6.0,0.0,3.0,5.0
chrX-103592126,chrX,CGG,0.000333,103592126,chrX-103592126,18.0,0.0,4.0,0.0,5.0,18.0,3.0,9.0,4.0,5.0


In [211]:
df_dar_dms.set_index(dms_stat_cov_selected.index)

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,fraca,fracb,effectsize,region_extended
dms,,,,,,,,,,,,,,
chr1-6526628,1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",0.030973,0.302326,-0.271352,chr1:6521617-6531633
chr1-6527055,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6522054-6532089
chr1-6527080,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6522054-6532089
chr1-6527082,1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",0.034884,0.138047,-0.103163,chr1:6522054-6532089
chr1-12577162,1,12577161,12577177,different,31.539541,11,"h:0,m:10","h:21,m:47","h:0.00,m:2.96","h:6.54,m:14.64",0.029586,0.211838,-0.182252,chr1:12572161-12582177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX-12195727,X,12195699,12195733,different,12.386906,21,"h:21,m:106","h:9,m:80","h:3.57,m:18.00","h:3.93,m:34.93",0.215620,0.388646,-0.173027,chrX:12190699-12200733
chrX-73794972,X,73794922,73794975,different,6.122430,28,"h:13,m:87","h:5,m:80","h:1.96,m:13.12","h:1.38,m:22.10",0.150830,0.234807,-0.083977,chrX:73789922-73799975
chrX-103592126,X,103592123,103592127,different,23.112939,4,"h:19,m:26","h:8,m:38","h:16.81,m:23.01","h:15.69,m:74.51",0.398230,0.901961,-0.503731,chrX:103587123-103597127


In [213]:
df_dms_dmr = df_dar_dms.set_index(dms_stat_cov_selected.index).merge(dms_stat_cov_selected,left_index=True,right_index=True,how='inner')

In [268]:
import pysam

dict_ct_count = {}

column =[]

for celltype in cell_types:
    tabix_file = pysam.TabixFile(f'/data2st1/junyi/merged_allcs/PFC/{celltype}.tsv.gz')
    region_data = []

    mc = 0
    cov = 0

    for row in tabix_file.fetch("chr1", 6527054, 6527089):
        columns = row.split('\t')  # Split into columns (adjust delimiter if needed)
        mc += int(columns[4])
        cov += int(columns[5])
        column.append(columns)

        dict_ct_count[celltype] = [mc, cov,mc/cov]


In [272]:
from multiprocessing import Pool
def mc_coverage_by_region(allcfiles:list,regions:list,ncpus=4):

    def process_celltype(allcfile,regions):
        tabix_file = pysam.TabixFile(allcfile)
        mc = 0
        cov = 0
        for region in regions:
            chr = region[0]
            start = region[1]
            end = region[2]
            for row in tabix_file.fetch(chr, start, end):
                columns = row.split('\t')
                mc += int(columns[4])
                cov += int(columns[5])

        return [mc, cov]
    # Create a pool of workers
    with Pool(processes=ncpus) as pool:  # Adjust number of processes as needed
        results = pool.starmap(process_celltype, [(allcfile,regions) for allcfile in allcfiles])
    results_dict = dict(zip(allcfiles,results))

    return results_dict

In [276]:
df_dms_dmr['chr'] = 'chr'+df_dms_dmr['chr'].astype(str)

In [278]:
df_dms_dmr_selected =df_dms_dmr.drop_duplicates(subset=['chr','start','end'],keep='first')

In [280]:
df_dms_dmr_selected.head()

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,...,mc_Neuron,mc_Astro-Epen,mc_Vascular,mc_Immune,mc_OPC-Oligo,cov_Neuron,cov_Astro-Epen,cov_Vascular,cov_Immune,cov_OPC-Oligo
dms,,,,,,,,,,,,,,,,,,,,,
chr1-6526628,chr1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",...,19.0,0.0,0.0,0.0,0.0,49.0,9.0,3.0,1.0,12.0
chr1-6527055,chr1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",...,32.0,0.0,0.0,0.0,0.0,81.0,10.0,8.0,7.0,20.0
chr1-12577162,chr1,12577161,12577177,different,31.539541,11,"h:0,m:10","h:21,m:47","h:0.00,m:2.96","h:6.54,m:14.64",...,36.0,0.0,1.0,0.0,0.0,43.0,5.0,4.0,2.0,12.0
chr1-16878737,chr1,16878736,16878786,different,18.738365,26,"h:17,m:106","h:20,m:158","h:1.76,m:11.00","h:2.75,m:21.70",...,70.0,4.0,5.0,0.0,9.0,73.0,9.0,8.0,0.0,18.0
chr1-18098986,chr1,18098940,18099022,different,19.671089,37,"h:15,m:136","h:28,m:148","h:1.04,m:9.40","h:3.13,m:16.54",...,46.0,3.0,5.0,2.0,5.0,49.0,5.0,6.0,2.0,15.0


In [284]:
df_dms_dmr_selected[['chr','start','end']].head().values.tolist()[0]

['chr1', 6526617, 6526633]

In [285]:
allc_paths

['/data2st1/junyi/merged_allcs/PFC/Neuron.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Astro-Epen.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Vascular.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/Immune.tsv.gz',
 '/data2st1/junyi/merged_allcs/PFC/OPC-Oligo.tsv.gz']

In [291]:
df_dms_dmr_selected

,chr,start,end,state,scores,nsties,counta,couontb,pcta,pctb,...,mc_Neuron,mc_Astro-Epen,mc_Vascular,mc_Immune,mc_OPC-Oligo,cov_Neuron,cov_Astro-Epen,cov_Vascular,cov_Immune,cov_OPC-Oligo
dms,,,,,,,,,,,,,,,,,,,,,
chr1-6526628,chr1,6526617,6526633,different,28.960890,6,"h:4,m:3","h:5,m:34","h:1.77,m:1.33","h:3.88,m:26.36",...,19.0,0.0,0.0,0.0,0.0,49.0,9.0,3.0,1.0,12.0
chr1-6527055,chr1,6527054,6527089,different,13.846013,14,"h:7,m:11","h:12,m:29","h:1.36,m:2.13","h:4.04,m:9.76",...,32.0,0.0,0.0,0.0,0.0,81.0,10.0,8.0,7.0,20.0
chr1-12577162,chr1,12577161,12577177,different,31.539541,11,"h:0,m:10","h:21,m:47","h:0.00,m:2.96","h:6.54,m:14.64",...,36.0,0.0,1.0,0.0,0.0,43.0,5.0,4.0,2.0,12.0
chr1-16878737,chr1,16878736,16878786,different,18.738365,26,"h:17,m:106","h:20,m:158","h:1.76,m:11.00","h:2.75,m:21.70",...,70.0,4.0,5.0,0.0,9.0,73.0,9.0,8.0,0.0,18.0
chr1-18098986,chr1,18098940,18099022,different,19.671089,37,"h:15,m:136","h:28,m:148","h:1.04,m:9.40","h:3.13,m:16.54",...,46.0,3.0,5.0,2.0,5.0,49.0,5.0,6.0,2.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr19-58245548,chr19,58245545,58245606,different,96.390010,29,"h:5,m:30","h:10,m:149","h:0.51,m:3.07","h:1.76,m:26.19",...,20.0,1.0,2.0,0.0,1.0,21.0,7.0,9.0,2.0,8.0
chrX-12195700,chrX,12195699,12195733,different,12.386906,21,"h:21,m:106","h:9,m:80","h:3.57,m:18.00","h:3.93,m:34.93",...,38.0,0.0,3.0,1.0,2.0,41.0,10.0,3.0,7.0,5.0
chrX-73794972,chrX,73794922,73794975,different,6.122430,28,"h:13,m:87","h:5,m:80","h:1.96,m:13.12","h:1.38,m:22.10",...,6.0,5.0,0.0,2.0,3.0,26.0,6.0,0.0,3.0,5.0


In [286]:
df_dms_dmr_selected[['chr','start','end']].head().values.tolist()

[['chr1', 6526617, 6526633],
 ['chr1', 6527054, 6527089],
 ['chr1', 12577161, 12577177],
 ['chr1', 16878736, 16878786],
 ['chr1', 18098940, 18099022]]

In [287]:
from utils import mc_coverage_by_region
allc_paths = list(allc_table.values())
regions = df_dms_dmr_selected[['chr','start','end']].values.tolist()
results_dict = mc_coverage_by_region(allc_paths,regions,ncpus=5)

In [290]:
np.array(results_dict['/data2st1/junyi/merged_allcs/PFC/Neuron.tsv.gz'])

array([[ 90, 241],
       [211, 975],
       [191, 614],
       ...,
       [ 62, 943],
       [ 46, 107],
       [ 80, 230]])

In [267]:

df_count = pd.DataFrame(dict_ct_count)
df_count

,Neuron,Astro-Epen,Vascular,Immune,OPC-Oligo
0,211.00000,7.000000,3.000000,4.000000,4.00
1,975.00000,150.000000,121.000000,67.000000,200.00
2,0.21641,0.046667,0.024793,0.059701,0.02


In [258]:
pd.DataFrame(region_data).iloc[:,[4,5]].astype(int).sum(axis=0)

4     14933
5    197248
dtype: int64

In [215]:
df_dms_dmr.to_csv(f'{output_dir}/dms_dmr_stat.csv',index=False)

In [227]:
counts

'mc_Neuron'

In [133]:
chrom_filter = dms_ds['dms_p-values'] <0.05 # 创建布尔掩码


In [134]:
filtered_ds = dms_ds.where(chrom_filter, drop=True)

In [137]:
filtered_ds

<xarray.Dataset>
Dimensions:       (sample: 5, dms: 943, count_type: 2)
Coordinates:
  * count_type    (count_type) <U3 'mc' 'cov'
  * dms           (dms) <U15 'chr1-37820074' ... 'chrX-162898192'
    dms_chrom     (dms) <U5 dask.array<chunksize=(188,), meta=np.ndarray>
    dms_contexts  (dms) <U3 dask.array<chunksize=(188,), meta=np.ndarray>
    dms_p-values  (dms) float64 dask.array<chunksize=(188,), meta=np.ndarray>
    dms_pos       (dms) int64 dask.array<chunksize=(188,), meta=np.ndarray>
  * sample        (sample) <U10 'Neuron' 'Astro-Epen' ... 'Immune' 'OPC-Oligo'
Data variables:
    dms_da        (sample, dms, count_type) float64 dask.array<chunksize=(1, 188, 1), meta=np.ndarray>
    dms_da_frac   (sample, dms) float64 dask.array<chunksize=(1, 188), meta=np.ndarray>
    dms_residual  (sample, dms) float64 dask.array<chunksize=(1, 188), meta=np.ndarray>
Attributes:
    region_dim:          dms
    region_ds_location:  /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP
    chrom_size_path:     /data2st1/junyi/output/dmr/PFC/Nano_PFC_HIP/chrom_si...